# THE EASY WAY
importing package cv2 and using it to detect where are faces (works on multiple faces)

In [3]:
import tkinter as tk
import tkinter.ttk as ttk
import tkinter.font as fnt
from tkinter import filedialog
import cv2


In [4]:
face_cascade = cv2.CascadeClassifier('haarsascade_frontalface_default.xml')
path = "SampleVideo.mp4"

def AddVideoEvent():
    cap = cv2.VideoCapture(path)
    show_video(cap)

In [4]:
def show_video(cap):
    while True:
        # Read the frame
        _, img = cap.read()

        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Detect the faces
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        # Draw the rectangle around each face
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Display
        cv2.imshow('img', img)

        # Stop if escape key is pressed
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    
    # Release the VideoCapture object
    cap.release()
    cv2.destroyAllWindows()

In [5]:
AddVideoEvent()

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


## THE HARD WAY

now i will try to implement this myself

### 1.1 COLLECTING DATA
for my first try i will collect data out of videos i found online and i captured photos out of them and making labels out of them

In [5]:
import os
import time
import uuid
import cv2

In [6]:
IMAGES_PATH = os.path.join('Data','Images')
videos_paths = [os.path.join('DataVideos',f"Video{i}.mp4") for i in range(1,7)]
number_images = 10


In [23]:

for path in videos_paths:
    cap = cv2.VideoCapture(path)
    i = 0
    for imgnum in range(number_images * 4):
        
        ret, frame = cap.read()
        if i % 4 == 0:
            print('Collecting image {}'.format((imgnum//4))
            imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
            cv2.imwrite(imgname, frame)
            cv2.imshow('frame', frame)

        time.sleep(0.5)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        i += 1
    cap.release()
    cv2.destroyAllWindows()

SyntaxError: '(' was never closed (3776592053.py, line 8)

In [ ]:
!labelme

### 1.2 COLLECTING IMAGES OUT OF UTK DATABASE

i will use opencv library to detect faces out of my database and create labels to work with them later


In [19]:
import json
from PIL import Image

In [20]:
def create_labelme_json(label, points, image_path, image_height, image_width):
    points = [[int(x), int(y)] for x, y in points]
    
    return {
        "version": "5.3.1",
        "flags": {},
        "shapes": [
            {
                "label": label,
                "points": points,  # coordinates of the top-left and bottom-right corners
                "group_id": None,
                "shape_type": "rectangle",
                "flags": {}
            },
            # add more shapes if necessary
        ],
        "imagePath": image_path,
        "imageData": None,
        "imageHeight": image_height,
        "imageWidth": image_width
    }

def add_file_to_folder_json(folder, subfolder, filename, data):
    base_name = os.path.splitext(filename)[0]
    # Construct the new file name
    output_file = base_name + '.json'
    
    file_path = os.path.join(folder, subfolder, output_file)
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    with open(file_path, 'w') as f:
        json.dump(data, f)

def delete_from_folder(folder,subfolder,filename):
    file_path = os.path.join(folder, subfolder, filename)

    try:
        # Check if the file exists
        if os.path.exists(file_path):
            # Delete the file
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        else:
            print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error deleting file: {e}")

In [21]:

directoryImages = ".\\UTKData\\Images"
files = os.listdir(directoryImages)

In [22]:
def get_image_size(file_path):
    # Open the image file
    img = Image.open(file_path)
    # Get the width and height of the image
    width, height = img.size
    return width, height

def save_label(image_path):
    file_path = f".\\UTKData\\Images\\{image_path}"
    img = cv2.imread(file_path)
    width, height = get_image_size(file_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    # Draw the rectangle around each face
    if (len(faces) > 0):
        (x, y, w, h) = faces[0]
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
            
        my_json_file = create_labelme_json("face",[[x, y],[x + w, y + h]], file_path, height, width)
        # print(my_json_file)
        add_file_to_folder_json("UTKData","Labels",image_path,my_json_file)
    # cv2.destroyAllWindows()


In [23]:
print(files[0])

10_0_0_20170116201102149.jpg


In [24]:
for file in files:
    save_label(file)